# Create a heatmap of all annotated ions significantly related to gestational age

load libraries

In [1]:
library(gplots)
library(RColorBrewer)
library(scales)
library(viridis)
library(beeswarm)

Warning message:
“package ‘gplots’ was built under R version 3.6.2”

Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess


Warning message:
“package ‘scales’ was built under R version 3.6.2”
Loading required package: viridisLite


Attaching package: ‘viridis’


The following object is masked from ‘package:scales’:

    viridis_pal




load results from univariate analysis and hypergeometric testing

In [2]:
t <- read.csv('../HypergeometricTesting/output.csv', stringsAsFactors = F)

In [3]:
head(t)
dim(t)

,node,sig,p_neigh,sig_neigh,hits
,<int>,<chr>,<dbl>,<chr>,<chr>
1,48,False,1.0000000,False,
2,73,False,0.1450814,False,
3,74,False,1.0000000,False,
4,84,False,0.8336212,False,
5,100,False,1.0000000,False,
6,104,False,0.5866621,False,(CCMSLIB00000579927:CocamidoprpylBetaine)


[1] 9010    5

add manually annotated compounds

In [6]:
t$hits[which(t$node == '8491')] <- 'Quinaldic acid, manually annotated'
t$hits[which(t$node == '19842')] <- 'Imidazole propionate, manually annotated'

load results from LASSO regression

In [8]:
lasso <- read.csv('../LASSO/_coefs_imported.csv', comment.char = "", check.names = F)

In [9]:
length(lasso$'#OTU ID')

[1] 83

In [10]:
t$lasso <- rep('False',nrow(t))

In [11]:
t$lasso[which(t$node %in% lasso$'#OTU ID')] <- 'True'

In [12]:
t <- t[, c("node", "sig", "p_neigh", "sig_neigh","lasso", "hits")]

In [14]:
head(t[match(lasso$'#OTU ID',t$node),])
dim(t[match(lasso$'#OTU ID',t$node),])

,node,sig,p_neigh,sig_neigh,lasso,hits
,<int>,<chr>,<dbl>,<chr>,<chr>,<chr>
99,1037,False,1.000000000,False,True,
170,1574,True,0.070232734,False,True,
195,1764,True,0.003958164,True,True,
315,2698,False,0.173604216,False,True,
327,2710,True,0.003958164,True,True,
436,2825,True,0.306870606,False,True,


[1] 83  6

which ions either significantly correlate with gestational age, are overrepresented in significant neighbours or were selected by LASSO?

In [15]:
tsigs <- t[which(t$sig == 'True' | t$sig_neigh == 'True' | t$lasso == 'True'),]

In [16]:
head(tsigs)
dim(tsigs)

,node,sig,p_neigh,sig_neigh,lasso,hits
,<int>,<chr>,<dbl>,<chr>,<chr>,<chr>
18,309,True,7.764734e-02,False,False,
60,748,False,1.072255e-03,True,False,
61,764,False,1.869112e-11,True,False,
70,819,False,3.832410e-02,True,False,
91,1001,True,1.000000e+00,False,False,
99,1037,False,1.000000e+00,False,True,


[1] 1843    6

In [17]:
length(which(tsigs$sig == 'True'))

[1] 1459

In [19]:
length(which(tsigs$sig_neigh == 'True'))

[1] 617

In [20]:
length(which(tsigs$lasso == 'True'))

[1] 83

In [21]:
tsigs[which(tsigs$sig == 'True' & tsigs$sig_neigh == 'True' & tsigs$lasso == 'True'),]

,node,sig,p_neigh,sig_neigh,lasso,hits
,<int>,<chr>,<dbl>,<chr>,<chr>,<chr>
195,1764,True,0.003958164,True,True,
327,2710,True,0.003958164,True,True,
871,3341,True,0.004039273,True,True,(CCMSLIB00005464307:N1-ACETYLSPERMINE)
2267,5917,True,0.031604197,True,True,
2527,6730,True,0.004888264,True,True,
2641,7139,True,0.043721107,True,True,


which of those ions have a library match? (subtract 2, as 2 features were annotated manually)

In [22]:
dim(tsigs[which(tsigs$sig == 'True' & tsigs$hits != ''),])

[1] 74  6

In [24]:
write.table(tsigs[which(tsigs$sig == 'True' & tsigs$hits != ''),], 'AllLibraryMatches_UnivariateCorrelation.txt', sep = '\t', row.names = F)

In [23]:
tsigslib <- tsigs[which(tsigs$hits != ''),]

In [24]:
head(tsigslib)
dim(tsigslib)

,node,sig,p_neigh,sig_neigh,lasso,hits
,<int>,<chr>,<dbl>,<chr>,<chr>,<chr>
254,2270,True,1.545147e-05,True,False,(CCMSLIB00005464388:SPERMINE)
258,2315,True,7.519579e-02,False,False,(CCMSLIB00005464134:L-CARNITINE)
299,2637,True,8.143536e-01,False,False,(CCMSLIB00003138708:Spectral Match to Phe-Phe from NIST14)
303,2680,False,2.755157e-02,True,False,(CCMSLIB00005464316:O-ACETYLCARNITINE)
313,2695,True,1.138082e-01,False,False,(CCMSLIB00004720318:Gyromitrin)
333,2716,True,4.039273e-03,True,False,(CCMSLIB00005463985:SPERMIDINE)


[1] 100   6

In [27]:
write.table(tsigslib,'SignificantFeatures_withlibraryIDs.txt',sep = '\t', quote = F, row.names = F)

In [26]:
tsigslib_annot <- read.table('SignificantFeatures_withlibraryIDs_ChemicalClasses.txt', comment.char = "", check.names = F, sep = '\t', header = T)

In [28]:
head(tsigslib_annot)

,node,sig,p_neigh,sig_neigh,lasso,hits,Compound_Name_Nice,ChemicalClass,Notes
,<int>,<lgl>,<fct>,<lgl>,<lgl>,<fct>,<fct>,<fct>,<fct>
1,2270,TRUE,"1,55E+09",TRUE,FALSE,(CCMSLIB00005464388:SPERMINE),Spermine,Polyamine,
2,2315,TRUE,0.075195786,FALSE,FALSE,(CCMSLIB00005464134:L-CARNITINE),Carnitine,Carnitine,
3,2637,TRUE,0.814353623,FALSE,FALSE,(CCMSLIB00003138708:Spectral Match to Phe-Phe from NIST14),Phe-Phe,Dipeptide,
4,2680,FALSE,0.027551575,TRUE,FALSE,(CCMSLIB00005464316:O-ACETYLCARNITINE),Acetylcarnitine,Carnitine,
5,2695,TRUE,0.11380824,FALSE,FALSE,(CCMSLIB00004720318:Gyromitrin),,Amino acid,False positive could be amino acid
6,2716,TRUE,0.004039273,TRUE,FALSE,(CCMSLIB00005463985:SPERMIDINE),Spermidine,Polyamine,


In [29]:
lib <- read.table('GNPSLibraryHits.tsv', header = T, comment.char = "", quote = "", sep = '\t', check.names = F)

In [30]:
tsigslib_annot <- merge(tsigslib_annot,lib, by.x = 'node', by.y = '#Scan#', all.x = T)

In [31]:
dim(tsigslib_annot)

[1] 100  44

select only library hits with a cosine score of >= 0.9

In [75]:
tsigslib_annot <- tsigslib_annot[which(tsigslib_annot$MQScore >= 0.9),]

In [76]:
dim(tsigslib_annot)

[1] 53 44

import featuretable

In [32]:
ft <- read.csv('../../featuretable_blankfiltered_batchnormalised.tsv', sep = '\t', check.names = F)

select all 'significant' ions in the feature table 

In [77]:
intft <- ft[which(ft$'#OTU ID' %in% tsigslib_annot$node),]

In [78]:
dim(intft)

[1]  53 299

import metadata

In [79]:
md <- read.table('../../metadata.tsv', sep = '\t', comment.char = '', check.names = F, header = T)

import original 150 sample subset

In [80]:
md_150 <- read.csv('../../metadata_150.tsv', sep = '\t', check.names = F, stringsAsFactors = F)

In [82]:
md_150 <- md_150[-which(md_150$ATTRIBUTE_SampleType != 'Sample'),]

In [83]:
dim(md_150)

[1] 148  14

In [84]:
selfiles <- as.character(md$'#SampleID'[which(md$'#SampleID' %in% md_150$filename)])

In [85]:
rownames(intft) <- intft$'#OTU ID'

In [86]:
intftsel <- intft[,which(colnames(intft) %in% selfiles)]

In [88]:
intftsel <- intftsel[,order(md$'Gestationsalder'[match(colnames(intftsel),md$'#SampleID')])]

In [89]:
intftsel <- intftsel[order(as.character(tsigslib_annot$ChemicalClass[match(rownames(intftsel),tsigslib_annot$node)])),]

In [91]:
my_palette <- colorRampPalette(c("blue", "white","red"))(n = 30)

In [92]:
cols_row <- viridis(15)

In [93]:
cols_col <- brewer.pal(length(unique(as.character(tsigslib_annot$ChemicalClass[match(rownames(intftsel),tsigslib_annot$node)]))), 'Paired')

In [94]:
ga_cols <- cols_row[as.numeric(as.factor(md$'Gestationsalder'[match(colnames(intftsel),md$'#SampleID')]))]

In [95]:
chemclass_cols <- cols_col[as.numeric(as.factor(as.character(tsigslib_annot$ChemicalClass[match(rownames(intftsel),tsigslib_annot$node)])))]

In [96]:
rownames(intftsel) <- paste(rownames(intftsel), as.character(tsigslib_annot$Compound_Name_Nice[match(rownames(intftsel),tsigslib_annot$node)]), sep = '_')

In [98]:
pdf(file="Heatmap.pdf", width=20, height=15)
heatmap.2(as.matrix(intftsel), 
          cexCol = 0.5,
          cexRow = 1.3,
          scale="row", 
          margins = c(20, 30), 
          col = my_palette, 
          Rowv = TRUE, 
          Colv = FALSE, 
          tracecol = NA, 
          trace='none',
          density.info='none',
          ColSideColors= ga_cols, 
          RowSideColors= chemclass_cols, 
          dendrogram = 'none', 
          keysize = 0.8,
          key.title = '',
          key.par = list(cex = 1.5),
          key = TRUE) #
legend("left", fill = cols_col,
       legend = sort(as.character(unique(tsigslib_annot$ChemicalClass))),cex=1.5,horiz = F)
dev.off()

pdf 
  2